# 🧠 Stable Diffusion Colab - Factory Reset 대응 버전
본 노트북은 Stable Diffusion v1.5 기반 이미지 생성, img2img 기능, Hugging Face 로그인, Google Drive 자동 저장 등을 포함합니다.

In [ ]:
# ✅ 환경 초기화
!pip uninstall -y numpy transformers diffusers tensorflow jax jaxlib --quiet
!pip install numpy==1.26.4 --quiet
!pip install torch torchvision --quiet
!pip install transformers==4.39.3 diffusers==0.27.2 --quiet
!pip install gradio==3.50.2 huggingface_hub --quiet

In [ ]:
# ✅ Hugging Face 로그인
from huggingface_hub import login
login()

In [ ]:
# ✅ 모델 불러오기
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import gradio as gr
import numpy as np
import os
from datetime import datetime

pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
    use_auth_token=True
).to("cuda")

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
    use_auth_token=True
).to("cuda")

In [ ]:
# ✅ Google Drive 연동
from google.colab import drive
drive.mount('/content/drive')
SAVE_DIR = "/content/drive/MyDrive/SD_Generated_Images"
os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
# ✅ 이미지 생성 함수
def infer(prompt, init_image, strength):
    if init_image is not None:
        if isinstance(init_image, np.ndarray):
            init_image = Image.fromarray(init_image)
        elif not isinstance(init_image, Image.Image):
            raise ValueError("Invalid image type")
        init_image = init_image.resize((512, 512))
        with autocast("cuda"):
            result = pipe_img2img(prompt=[prompt], init_image=init_image, strength=strength, guidance_scale=7.5)
    else:
        with autocast("cuda"):
            result = pipe(prompt=[prompt], guidance_scale=7.5)
    images = result.images
    for i, img in enumerate(images):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        img.save(f"{SAVE_DIR}/generated_{timestamp}_{i}.png")
    return images

In [ ]:
# ✅ Gradio 인터페이스
with gr.Blocks() as demo:
    gr.Markdown("""# Stable Diffusion 이미지 생성기""")
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="프롬프트")
            init_image = gr.Image(label="기준 이미지", type="numpy")
            strength = gr.Slider(label="유지 강도", minimum=0.2, maximum=1.0, value=0.75)
            run_button = gr.Button("실행")
        with gr.Column():
            output = gr.Gallery(label="결과")
    run_button.click(fn=infer, inputs=[prompt, init_image, strength], outputs=output)
demo.launch(debug=True, share=True)